In [1]:
!pip install tensorflow
!pip install keras
!pip install opencv-python

  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
  Running setup.py install for tensorflow-gpu: started
  Running setup.py install for tensorflow-gpu: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [18 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Edu\AppData\Local\Temp\pip-install-k96bcuth\tensorflow-gpu_a783ae5f9181401496396a9f6066609a\setup.py", line 37, in <module>
      raise Exception(TF_REMOVAL_WARNING)
  Exception:
  
  The "tensorflow-gpu" package has been removed!
  
  Please install "tensorflow" instead.
  
  Other than the name, the two packages have been identical
  since TensorFlow 2.1, or roughly since Sep 2019. For more
  information, see: pypi.org/project/tensorflow-gpu
  
  
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tensorflow-gpu
  error: subprocess-exited-with-error
  
  Running setup.py install for tensorflow-gpu did not r

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import cv2
import os
import numpy as np
from tqdm import tqdm
import time

In [3]:
root_path = "C:\\Users\\Edu\\Desktop\\CANCER"
image_size = (50, 50)

In [4]:
def cargar_imagenes(root_path, image_size):
    X = []
    y = []
    
    for folder in os.listdir(root_path):
        folder_path = os.path.join(root_path, folder)
        if not os.path.isdir(folder_path):
            continue
        
        for subfolder in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, subfolder)
            if not os.path.isdir(subfolder_path):
                continue
            
            label = int(subfolder)  # Convertir el nombre de la subcarpeta en etiqueta (0 o 1)
            
            for file_name in tqdm(os.listdir(subfolder_path)):
                img_path = os.path.join(subfolder_path, file_name)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                
                if img is None:
                    print(f"Error al leer la imagen: {img_path}")
                    continue
                
                img = cv2.resize(img, image_size)
                img_array = img_to_array(img)
                
                X.append(img_array)
                y.append(label)
    
    X = np.array(X)
    y = np.array(y)
    
    print(f"Se han cargado {len(X)} imágenes en total")
    print(f"{np.sum(y)} imágenes son positivas y {len(y) - np.sum(y)} son negativas")
    
    return X, y

In [5]:
# Conjuntos de entrenamiento, validación y prueba
def dividir_datos(X, y):
    X_entrenamiento, X_temp, y_entrenamiento, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)  # Divide los datos en entrenamiento y temporal (para validación y prueba)
    X_validacion, X_prueba, y_validacion, y_prueba = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # Divide los datos temporales en validación y prueba
    
    print(f"Entrenamiento: {len(X_entrenamiento)} imágenes, {np.sum(y_entrenamiento)} positivas")
    print(f"Validación: {len(X_validacion)} imágenes, {np.sum(y_validacion)} positivas")
    print(f"Prueba: {len(X_prueba)} imágenes, {np.sum(y_prueba)} positivas")
    
    return X_entrenamiento, X_validacion, X_prueba, y_entrenamiento, y_validacion, y_prueba  # Devuelve los conjuntos de datos


In [6]:
# Modelo de red neuronal convolucional
def crear_modelo():
    model = Sequential()  # Inicia un modelo secuencial
    model.add(Conv2D(32, kernel_size=3, activation="relu", input_shape=(image_size[0], image_size[1], 1)))  # Añade una capa convolucional que tiene 32 filtros
    model.add(Conv2D(64, kernel_size=3, activation="relu"))  # Añade otra capa convolucional con 64 filtros
    model.add(Flatten())  # Aplana la salida de la última capa convolucional
    model.add(Dense(1, activation="sigmoid"))  # Añade una capa densa de salida con 1 neurona
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Compila el modelo con el optimizador Adam, la pérdida de entropía cruzada binaria y la precisión como métricas
    return model  # Retorna el modelo creado

In [7]:
class TimingCallback(Callback): #nos da datos de progreso y tiempos
    def on_train_begin(self, logs={}):
        self.start_time = time.time()

    def on_epoch_end(self, batch, logs={}):
        elapsed_time = time.time() - self.start_time
        print(f"Tiempo transcurrido: {elapsed_time} segundos")


In [9]:
# Cargar las imágenes
X, y = cargar_imagenes(root_path, image_size)

# Dividir los datos
X_entrenamiento, X_validacion, X_prueba, y_entrenamiento, y_validacion, y_prueba = dividir_datos(X, y)

# Crear y entrenar el modelo
model = crear_modelo()
print("Comenzando el entrenamiento...")
cb = TimingCallback()
model.fit(X_entrenamiento, y_entrenamiento, validation_data=(X_validacion, y_validacion), epochs=5, callbacks=[cb])
print("Entrenamiento finalizado")

# Evaluar el modelo
print("Evaluando el modelo...")
loss, accuracy = model.evaluate(X_prueba, y_prueba)
print(f"Loss: {loss}\nAccuracy: {accuracy}")


100%|████████████████████████████████████████████████████████████████████████████████| 70/70 [00:00<00:00, 6171.98it/s]


Se han cargado 277524 imágenes en total
78786 imágenes son positivas y 198738 son negativas
Entrenamiento: 194266 imágenes, 55211 positivas
Validación: 41629 imágenes, 11905 positivas
Prueba: 41629 imágenes, 11670 positivas
Comenzando el entrenamiento...
Epoch 1/5
6071/6071 [==============================] - 146s 24ms/step - loss: 0.8116 - accuracy: 0.7478 - val_loss: 0.4887 - val_accuracy: 0.7763
Epoch 2/5
6071/6071 [==============================] - 170s 28ms/step - loss: 0.4900 - accuracy: 0.7733 - val_loss: 0.4939 - val_accuracy: 0.7757
Epoch 3/5
6071/6071 [==============================] - 178s 29ms/step - loss: 0.4770 - accuracy: 0.7800 - val_loss: 0.4776 - val_accuracy: 0.7720
Epoch 4/5
6071/6071 [==============================] - 163s 27ms/step - loss: 0.4596 - accuracy: 0.7901 - val_loss: 0.4979 - val_accuracy: 0.7780
Epoch 5/5
6071/6071 [==============================] - 175s 29ms/step - loss: 0.4419 - accuracy: 0.8005 - val_loss: 0.4940 - val_accuracy: 0.7667
Entrenamiento f